In [1]:
# NBA Shot Charts

In [2]:
%config IPCompleter.greedy=True

# Import the dependencies.
import json
import re
import time

import numpy as np
import pandas as pd
import requests

In [3]:
# The request files to verify its structure.
base_url = "https://stats.nba.com/stats/"

headers = {
    "Host": "stats.nba.com",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "x-nba-stats-origin": "stats",
    "x-nba-stats-token": "true",
    "Connection": "keep-alive",
    "Referer": "https://stats.nba.com/",
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",
}

measure_type = {"base": "Base", "advanced": "Advanced"}

stat_modes_depth = {
    "Per100Possessions": "Per100Possessions",
    "Totals": "Totals",
    "Per36": "Per36",
    "PerGame": "PerGame",
}

season_list = [
    "1996-97",
    "1997-98",
    "1998-99",
    "1999-00",
    "2000-01",
    "2001-02",
    "2002-03",
    "2003-04",
    "2004-05",
    "2005-06",
    "2006-07",
    "2007-08",
    "2008-09",
    "2009-10",
    "2010-11",
    "2011-12",
    "2012-13",
    "2013-14",
    "2014-15",
    "2015-16",
    "2016-17",
    "2017-18",
    "2018-19",
    "2019-20",
]

season_id = "2019-20"
team_id = 1610612737
player_id = 1629630
per_mode = "PerGame"

shot_chart_nbastats_api = f"{base_url}shotchartdetail?AheadBehind=&CFID=33&CFPARAMS=2019-20&ClutchTime=&Conference=&ContextFilter=&ContextMeasure=FGA&DateFrom=&DateTo=&Division=&EndPeriod=10&EndRange=28800&GROUP_ID=&GameEventID=&GameID=&GameSegment=&GroupID=&GroupMode=&GroupQuantity=5&LastNGames=0&LeagueID=00&Location=&Month=0&OnOff=&OpponentTeamID=0&Outcome=&PORound=0&Period=0&PlayerID={player_id}&PlayerID1=&PlayerID2=&PlayerID3=&PlayerID4=&PlayerID5=&PlayerPosition=&PointDiff=&Position=&RangeType=0&RookieYear=&Season={season_id}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StartPeriod=1&StartRange=0&StarterBench=&TeamID=0&VsConference=&VsDivision=&VsPlayerID1=&VsPlayerID2=&VsPlayerID3=&VsPlayerID4=&VsPlayerID5=&VsTeamID="

# json response
response = requests.get(url=shot_chart_nbastats_api, headers=headers).json()
# pretty printing data
pretty_data = json.dumps(response, indent=4)
print(pretty_data)
# pulling the headers we want to experiment with
stat_header = response["resultSets"][0]["headers"]
print(stat_header)
# pulling just the data we want
body_data = response["resultSets"][0]["rowSet"][0]
print(body_data)

{
    "resource": "shotchartdetail",
    "parameters": {
        "LeagueID": "00",
        "Season": "2019-20",
        "SeasonType": "Regular Season",
        "TeamID": 0,
        "PlayerID": 1629630,
        "GameID": null,
        "Outcome": null,
        "Location": null,
        "Month": 0,
        "SeasonSegment": null,
        "DateFrom": null,
        "DateTo": null,
        "OpponentTeamID": 0,
        "VsConference": null,
        "VsDivision": null,
        "Position": null,
        "RookieYear": null,
        "GameSegment": null,
        "Period": 0,
        "LastNGames": 0,
        "ClutchTime": null,
        "AheadBehind": null,
        "PointDiff": null,
        "RangeType": 0,
        "StartPeriod": 1,
        "EndPeriod": 10,
        "StartRange": 0,
        "EndRange": 28800,
        "ContextFilter": "SEASON_YEAR='2019-20'",
        "ContextMeasure": "FGA"
    },
    "resultSets": [
        {
            "name": "Shot_Chart_Detail",
            "headers": [
          

In [4]:
# transform contents into dataframe
results = response["resultSets"][0]
headers = stat_header
rows = results["rowSet"]
nba_shot_chart_df = pd.DataFrame(rows)
nba_shot_chart_df.columns = headers

# write to csv file
nba_shot_chart_df.to_csv("nba-shot-chart.csv", index=False)

In [5]:
nba_shot_chart_df["MAKE"] = nba_shot_chart_df.EVENT_TYPE.apply(
    lambda x: 0 if "Missed" in x else 1
)
nba_shot_chart_df["ZONE"] = nba_shot_chart_df.SHOT_TYPE.apply(
    lambda x: 2 if "2PT" in x else 3
)

In [6]:
nba_shot_chart_processed_df = nba_shot_chart_df[
    ["PLAYER_NAME", "LOC_X", "LOC_Y", "MAKE", "ZONE"]
]
nba_shot_chart_processed_df.columns = ["name", "x", "y", "make", "zone"]

In [7]:
nba_shot_chart_processed_df.head()

,name,x,y,make,zone
0,Ja Morant,-27,5,1,2
1,Ja Morant,18,11,1,2
2,Ja Morant,3,28,1,2
3,Ja Morant,-14,22,1,2
4,Ja Morant,-2,21,0,2


In [8]:
# write to csv file
nba_shot_chart_processed_df.to_csv("nba-shot-chart-processed.csv", index=False)

In [9]:
# write to json file
nba_shot_chart_processed_df.to_json("nba-shot-chart-processed.json", orient="values")